In [2]:
import requests
from PIL import Image
from io import BytesIO
import re


In [3]:
url = f"https://api.steampowered.com/ISteamApps/GetAppList/v2"
response = requests.get(url)
apps = response.json()['applist']['apps']
# print(len(apps))

In [54]:
def parse_requirements_string(req_string):
    req_dict = {}
    req_dict['req'] = req_string
    return req_dict
    os_match = re.search(r'OS:</strong>\s+(.+?)<br>', req_string)
    if os_match:
        req_dict['OS'] = os_match.group(1)
    
    processor_match = re.search(r'Processor:</strong>\s+(.+?)<br>', req_string)
    if processor_match:
        req_dict['Processor'] = processor_match.group(1)
    
    memory_match = re.search(r'Memory:</strong>\s+(.+?)<br>', req_string)
    if memory_match:
        req_dict['Memory'] = memory_match.group(1)
    
    graphics_match = re.search(r'Graphics:</strong>\s+(.+?)<br>', req_string)
    if graphics_match:
        req_dict['Graphics'] = graphics_match.group(1)
    
    storage_match = re.search(r'Storage:</strong>\s+(.+?) available space<br></li>', req_string)
    if storage_match:
        req_dict['Storage'] = storage_match.group(1)
    
    return req_dict


In [55]:
cpuList = {'-1'}
def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)
def display_game_thumbnail(appid):
    url = f"https://store.steampowered.com/api/appdetails?appids={appid}"
    response = requests.get(url)
    data = response.json()
    if not data:
        return '-1'
    # print(data)
    l = [str(appid), 'data', 'pc_requirements', 'minimum']
    for ll in l:
        if ll in data:
            data = data[ll]
        else:
            return '-1'
        
    # print(data)
    minimum_requirements = data
    return parse_requirements_string(minimum_requirements)
    minimum_requirements = remove_html_tags(minimum_requirements)
    pattern = r"Processor:\s*(.*?)\s*(?:Memory:|Graphics:|Storage:|$)"
    processor_match = re.search(pattern, minimum_requirements)

    if processor_match:
        processor_info = processor_match.group(1)
        # print(processor_info)
        return processor_info
    else:
        return '-1'
            # print("未找到处理器信息")
        # print(minimum_requirements)
# display_game_thumbnail(774171)
# idx = 0
# for app in apps:
#     idx += 1
#     if idx % 100 == 0:
#         print(cpuList)
#     cpuList.add(display_game_thumbnail(app['appid']))



In [56]:
print(display_game_thumbnail(774171))

{'req': '<strong>Minimum:</strong><br><ul class="bb_ul"><li><strong>OS:</strong> Windows 7 64bit or later<br></li><li><strong>Processor:</strong> Intel Core™ Duo or faster<br></li><li><strong>Memory:</strong> 2 GB RAM<br></li><li><strong>Graphics:</strong> DirectX® 9 Compatible Graphics Card<br></li><li><strong>Storage:</strong> 2 GB available space</li></ul>'}


In [57]:
def display_game_thumbnail(appid):
    url = f"https://store.steampowered.com/api/appdetails?appids={appid}"
    response = requests.get(url)
    data = response.json()
    t_url = ""
    if str(appid) in data:
        game_data = data[str(appid)]
        if game_data["success"]:
            t_url = game_data["data"]["capsule_imagev5"]
            print(t_url)
            
    l = [str(appid), 'data', 'pc_requirements', 'minimum']
    for ll in l:
        if ll in data:
            data = data[ll]
        else:
            return '-1'
        
    # print(data)
    minimum_requirements = data
    d = parse_requirements_string(minimum_requirements)
    # d['t_url'] = t_url
    return d
            # if thumbnail_url:
            #     response = requests.get(thumbnail_url)
            #     image_data = response.content
            #     image = Image.open(BytesIO(image_data))
            #     image.show()
            #     return

    # print("未找到游戏缩略图")

In [58]:
def search_games(query):
    games = []
    for app in apps:
        if query.lower() in app["name"].lower():
            games.append(app['appid'])

    return games

# search_results = search_games("Hogwarts Legacy")
search_results = [588650, 977950, 431960, 774171, 952040]
li = []
for i in search_results:
    print(display_game_thumbnail(i))
# print(li)


https://cdn.akamai.steamstatic.com/steam/apps/588650/capsule_184x69.jpg?t=1678188017
https://cdn.akamai.steamstatic.com/steam/apps/977950/capsule_184x69.jpg?t=1680659451
https://cdn.akamai.steamstatic.com/steam/apps/431960/capsule_184x69.jpg?t=1665921297
https://cdn.akamai.steamstatic.com/steam/apps/774171/capsule_184x69.jpg?t=1686650225
https://cdn.akamai.steamstatic.com/steam/apps/952040/capsule_184x69.jpg?t=1680573530
[{'req': '<strong>Minimum:</strong><br><ul class="bb_ul"><li><strong>OS:</strong> Windows 7+<br></li><li><strong>Processor:</strong> Intel i5+<br></li><li><strong>Memory:</strong> 2 GB RAM<br></li><li><strong>Graphics:</strong> Nvidia 450 GTS / Radeon HD 5750 or better<br></li><li><strong>Storage:</strong> 500 MB available space<br></li><li><strong>Additional Notes:</strong> DirectX 9.1+ or OpenGL 3.2+</li></ul>'}, {'req': '<strong>Minimum:</strong><br><ul class="bb_ul"><li><strong>OS:</strong> Windows 7 or later<br></li><li><strong>Memory:</strong> 2 GB RAM<br></li><l

In [51]:
import csv

def save_dict_to_csv(dict_list, filename):
    fieldnames = ['OS', 'Processor', 'Memory', 'Graphics', 'Storage'] #, 't_url']
    
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        
        for item in dict_list:
            print(item)
            writer.writerow(item)


In [52]:
save_dict_to_csv(li, 'l.csv')

{'OS': 'Windows 7+', 'Processor': 'Intel i5+', 'Memory': '2 GB RAM', 'Graphics': 'Nvidia 450 GTS / Radeon HD 5750 or better', 'Storage': '500 MB'}
{'OS': 'Windows 7 or later', 'Memory': '2 GB RAM', 'Graphics': 'Intel Graphics 4000, 2GB VRAM'}
{'OS': 'Windows 7 (with Aero), 8.1, 10, 11', 'Processor': '1.66 GHz Intel i5 or equivalent', 'Memory': '1024 MB RAM', 'Graphics': 'HD Graphics 4000 or above', 'Storage': '512 MB'}
